In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.6 MB/s eta 0:00:00


In [7]:
#device = torch.device('cuda' if torch.cuda.is_available() else "cpu")


In [4]:
import torch
#rom transformers import AutoImageProcessor, AutoModel
from PIL import Image
import faiss
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import json

In [ ]:
#load the model and processor


In [ ]:
## où trouver filenames ? Essayer de trouver une manière pour associer filenames à embeddings

def index_map_tensor(output_dir, embeddings, filenames):
    """
    Args:
        output_dir (str): where to save index + mapping
        embeddings (torch.Tensor): shape (N, D)
        filenames (list[str]): list of file/image names corresponding to embeddings
    """
    d = embeddings.shape[1]  # embedding dimension, e.g. 768
    index = faiss.IndexFlatL2(d)

    # Convert to float32 numpy (required by FAISS)
    all_vectors = embeddings.cpu().numpy().astype("float32")

    # Normalize if you want cosine similarity (optional)
    faiss.normalize_L2(all_vectors)

    # Add to FAISS index
    index.add(all_vectors)

    # Build mapping
    index_to_path = {i: os.path.basename(fname) for i, fname in enumerate(filenames)}

    # Save FAISS index
    faiss.write_index(index, os.path.join(output_dir, "objects_vector.index"))

    # Save JSON mapping
    with open(os.path.join(output_dir, "index_map_path.json"), "w") as f:
        json.dump(index_to_path, f)

    return index, index_to_path

In [ ]:
#Source : https://medium.com/aimonks/image-similarity-with-dinov2-and-faiss-741744bc5804
#Define a function that normalizes embeddings and add them to the index
def add_vector_to_index(embedding, index):
    #convert embedding to numpy
    vector = embedding.detach().cpu().numpy()
    #Convert to float32 numpy
    vector = np.float32(vector)
    #Normalize vector: important to avoid wrong results when searching
    faiss.normalize_L2(vector)
    #Add to index
    index.add(vector)

In [ ]:
# Path where your embeddings are stored
emb_folder = "path/to/pt_embeddings"

# Sort to ensure consistent ordering
pt_files = sorted([f for f in os.listdir(emb_folder) if f.endswith(".pt")])

embeddings = []
for f in pt_files:
    emb = torch.load(os.path.join(emb_folder, f))  # Load tensor
    emb = emb.squeeze(0)  # Make sure shape is (dim,), not (1, dim)
    embeddings.append(emb)

# Stack into a single tensor (N, dim)
embeddings = torch.stack(embeddings, dim=0)

In [ ]:


def index_map_file(output_dir, embeddings)

index = faiss.IndexFlatL2(768)

    all_vectors = []
    index_to_path = {}

    for idx, path in enumerate(tqdm(embeddings, desc="Loading embeddings")):
        vec = torch.load(path)  # shape [768]
        vec = vec.numpy().astype("float32")
        all_vectors.append(vec)

        # Save mapping
        index_to_path[idx] = os.path.basename(path)

    all_vectors = np.stack(all_vectors)

    faiss.normalize_L2(all_vectors)
    index.add(all_vectors)

    # Save FAISS index
    faiss.write_index(index, os.path.join(experiment_dir, "objects_vector.index"))

    # Save JSON mapping
    with open(os.path.join(experiment_dir, "index_map_path.json"), 'w') as f:
        json.dump(index_to_path, f)

    return index, index_to_path

## Cropud objets

In [5]:
EXPERIMENTS = '/content/drive/MyDrive/dataHN/experiments'
DATA = '/content/drive/MyDrive/dataHN/data'

In [6]:
experiment_dir = os.path.join(EXPERIMENTS, 'faiss_similarity')
os.makedirs(experiment_dir, exist_ok=True)

results_dir = os.path.join(experiment_dir, "results")
os.makedirs(results_dir, exist_ok=True)

In [7]:
obj_embeddings_dir = os.path.join(DATA, 'embeddings', 'dino3_obj_crops_gray')
obj_embeddings_list = [os.path.join(obj_embeddings_dir, e) for e in os.listdir(obj_embeddings_dir)]
obj_img_dir = os.path.join(DATA,'processed', 'g-dino_obj_crops')

In [8]:
doc_embeddings_dir = os.path.join(DATA, 'embeddings', 'dino3_doc_crops_gray')
doc_embeddings_list = [os.path.join(doc_embeddings_dir, e) for e in os.listdir(doc_embeddings_dir)]
doc_img_dir = os.path.join(DATA,'processed','yolo_doc_crops')

In [9]:
embedding = torch.load(obj_embeddings_list[0])
print(embedding.shape)
embedding = torch.load(doc_embeddings_list[0])
print(embedding.shape)

torch.Size([768])
torch.Size([768])


In [10]:
liste_embeddings = obj_embeddings_list + doc_embeddings_list

In [37]:
obj_id_to_path = {i: os.path.splitext(os.path.basename(obj_embeddings_list[i]))[0] for i in range(len(obj_embeddings_list))}
doc_id_to_path = {i: os.path.splitext(os.path.basename(doc_embeddings_list[i]))[0] for i in range(len(doc_embeddings_list))}

In [39]:
with open(os.path.join(results_dir, 'obj_index_file_map.json'), "w") as f:
    json.dump(obj_id_to_path, f)

with open(os.path.join(results_dir, 'doc_index_file_map.json'), "w") as f:
    json.dump(doc_id_to_path, f)

In [15]:
len(obj_embeddings_list)

2127

In [16]:
# Objets stack embeddings
stacked_path = os.path.join(DATA, 'embeddings', 'dino3_stacked', 'obj_embeddings.pt')

if os.path.exists(stacked_path):
    obj_embeddings = torch.load(stacked_path)  # tensor (N, 768)
else:
    all_embeddings = []
    for path in tqdm(obj_embeddings_list):
        emb = torch.load(path)                  # [768]
        all_embeddings.append(emb.unsqueeze(0)) # [1, 768]

    obj_embeddings = torch.cat(all_embeddings, dim=0)  # tensor (N, 768)
    torch.save(obj_embeddings, stacked_path)

100%|██████████| 2127/2127 [00:09<00:00, 231.43it/s]


In [17]:
# Docs stack path
stacked_path = os.path.join(DATA, 'embeddings', 'dino3_stacked', 'doc_embeddings.pt')

if os.path.exists(stacked_path):
    doc_embeddings = torch.load(stacked_path)  # tensor (N, 768)
else:
    all_embeddings = []
    for path in tqdm(doc_embeddings_list):
        emb = torch.load(path)                  # [768]
        all_embeddings.append(emb.unsqueeze(0)) # [1, 768]

    doc_embeddings = torch.cat(all_embeddings, dim=0)  # tensor (N, 768)
    torch.save(doc_embeddings, stacked_path)

100%|██████████| 7705/7705 [09:39<00:00, 13.30it/s]


## Index Faiss

In [18]:
print(obj_embeddings.shape)     # torch.Size([N, 768])
print(doc_embeddings.shape)     # torch.Size([N, 768])

torch.Size([2127, 768])
torch.Size([7705, 768])


In [ ]:
# # ---------------------------
# # 1. Load embeddings
# # ---------------------------
# def load_embeddings(list_paths):
#     """
#     Loads a list of .pt embeddings and stacks into (N, D) numpy float32 array.
#     Returns (embeddings, list_of_paths).
#     """
#     all_embs = []
#     for path in list_paths:
#         emb = torch.load(path)  # shape [D] or [1, D]
#         if emb.ndim == 1:
#             emb = emb.unsqueeze(0)
#         all_embs.append(emb)
#     embs = torch.cat(all_embs, dim=0).numpy().astype(np.float32)
#     return embs, list_paths

In [ ]:
# # Example: replace with your actual file lists
# docs_list = ["docs_emb/doc1.pt", "docs_emb/doc2.pt", "..."]
# objs_list = ["objs_emb/obj1.pt", "objs_emb/obj2.pt", "..."]

# embs_docs, paths_docs = load_embeddings(docs_list)
# embs_objs, paths_objs = load_embeddings(objs_list)

  # number of neighbors


In [ ]:
dim = 768
k = 20


In [20]:
if isinstance(doc_embeddings, torch.Tensor):
    doc_embeddings = doc_embeddings.cpu().numpy().astype(np.float32)
if isinstance(obj_embeddings, torch.Tensor):
    obj_embeddings = obj_embeddings.cpu().numpy().astype(np.float32)


faiss.normalize_L2(doc_embeddings)
faiss.normalize_L2(obj_embeddings)

index_docs = faiss.IndexFlatIP(dim)
index_objs = faiss.IndexFlatIP(dim)

index_docs.add(doc_embeddings)
index_objs.add(obj_embeddings)

print(f"Nombre de vecteurs dans index docs : {index_docs.ntotal}")
print(f"Nombre de vecteurs dans index obj: {index_objs.ntotal}")

Nombre de vecteurs dans index docs : 7705
Nombre de vecteurs dans index obj: 2127


In [21]:
faiss.write_index(index_objs, os.path.join(experiment_dir, "index_objs_cosine.faiss"))
faiss.write_index(index_docs, os.path.join(experiment_dir, "index_docs_cosine.faiss"))

In [22]:
# ---------------------------
# 3. Run searches
# ---------------------------
# Within-corpus
D_docs, I_docs = index_docs.search(doc_embeddings, k)
D_objs, I_objs = index_objs.search(obj_embeddings, k)

# Cross-corpus
D_doc2obj, I_doc2obj = index_objs.search(doc_embeddings, k)
D_obj2doc, I_obj2doc = index_docs.search(obj_embeddings, k)

In [29]:
obj_filenames = [os.path.basename(filepath).split('.')[0] for filepath in obj_embeddings_list]
doc_filenames = [os.path.basename(filepath).split('.')[0] for filepath in doc_embeddings_list]
paths_objs = [os.path.join(obj_img_dir, f'{filename}.jpg') for filename in obj_filenames]
paths_docs = [os.path.join(doc_img_dir, f'{filename}.jpg') for filename in doc_filenames]
print(paths_objs[0])


/content/drive/MyDrive/dataHN/data/processed/g-dino_obj_crops/GazetteDrouot_22972746_11_02.jpg


In [43]:
# ---------------------------
# 4. Convert results to DataFrames
# ---------------------------
def results_to_df(I, D, query_paths, corpus_paths, prefix="top"):
    rows = []
    for q_idx, query_path in enumerate(query_paths):
        row = {"query": os.path.basename(query_path)}  # store filename only for query
        for rank, (idx, dist) in enumerate(zip(I[q_idx], D[q_idx])):
            # store filename only
            row[f"{prefix}{rank}_path"] = os.path.basename(corpus_paths[idx])
            row[f"{prefix}{rank}_score"] = float(dist)
        rows.append(row)
    return pd.DataFrame(rows)


In [44]:
df_docs_docs = results_to_df(I_docs, D_docs, paths_docs, paths_docs, prefix="docs")
df_objs_objs = results_to_df(I_objs, D_objs, paths_objs, paths_objs, prefix="objs")
df_doc2obj = results_to_df(I_doc2obj, D_doc2obj, paths_docs, paths_objs, prefix="obj")
df_obj2doc = results_to_df(I_obj2doc, D_obj2doc, paths_objs, paths_docs, prefix="doc")

In [48]:


df_docs_docs.to_csv(os.path.join(results_dir, "docs_within_docs.csv"), index=False)
df_objs_objs.to_csv(os.path.join(results_dir,"objs_within_objs.csv"), index=False)
df_doc2obj.to_csv(os.path.join(results_dir,"docs_to_objs.csv"), index=False)
df_obj2doc.to_csv(os.path.join(results_dir,"objs_to_docs.csv"), index=False)

In [ ]:
def image_similarity_search(embeddings, index, image_name_list, experiment_dir, top_k=6):

    result_dict = {
        'top0_similar': [],
        'top1_similar': [],
        'top2_similar': [],
        'top3_similar': [],
        'top4_similar': [],
        'top5_similar': []
    }

    for embed in embeddings:
        embed = embed.reshape(1, -1)
        faiss.normalize_L2(embed)
        distances, ann = index.search(embed, k=top_k)

        for k in range(top_k):
            idx = ann[0][k]

            result_dict[f'top{str(k)}_similar'].append(image_name_list[idx])

    df = pd.DataFrame.from_dict(result_dict)
    df.to_csv(os.path.join(experiment_dir, f'obj_results.csv'), index=None)

In [ ]:
image_paths = [os.path.join(obj_img_dir, img) for img in os.listdir(obj_img_dir)]

In [ ]:
image_similarity_search(all_vectors, index, image_paths, experiment_dir)

In [ ]:
df = pd.read_csv(os.path.join(experiment_dir, 'obj_results.csv'))

In [ ]:
df.

,top0_similar,top1_similar,top2_similar,top3_similar,top4_similar,top5_similar
0,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
1,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
2,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
3,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
4,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
...,...,...,...,...,...,...
2122,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
2123,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
2124,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...
2125,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...,/content/drive/MyDrive/dataHN/data/processed/g...


In [ ]:
## Trouver photos identiques

In [ ]:
'''
hash-based exact duplicate detection or exact vector matching.
Each embedding is converted to a hashable form (in our case, .tobytes()), and we use a dictionary to group identical hashes.
This detects exact duplicates, meaning embeddings that are bitwise identical.
'''

# Load all embeddings into a numpy array
all_vectors = []
for path in tqdm(embeddings, desc="Loading embeddings"):
    vec = torch.load(path)  # shape [768]
    all_vectors.append(vec.numpy() if torch.is_tensor(vec) else vec)

all_vectors = np.stack(all_vectors)  # shape [num_images, 768]

# Create a dictionary to group duplicates
duplicates = defaultdict(list)

# Use hashes of the vectors for exact matching
for i, vec in enumerate(all_vectors):
    # Convert vector to bytes to hash exactly
    key = vec.tobytes()
    duplicates[key].append(embeddings[i])

# Print groups of duplicates
for group in duplicates.values():
    if len(group) > 1:  # only groups with true duplicates
        print("Identical embeddings found:", group)


Loading embeddings: 100%|██████████| 2127/2127 [00:08<00:00, 251.43it/s]


Identical embeddings found: ['/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_5424025_05_01.pt', '/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_5424025_04_01.pt']
Identical embeddings found: ['/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_15511693_01_01.pt', '/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_16465688_01_01.pt']
Identical embeddings found: ['/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_23842390_02_01.pt', '/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_22210569_02_01.pt']
Identical embeddings found: ['/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_18882041_01_01.pt', '/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/GazetteDrouot_20835052_01_01.pt', '/content/drive/MyDrive/dataHN/experiments/dinov3_embeddings/objets/Ga

In [ ]:
from PIL import Image
import math

def create_contact_sheet(image_paths, sheet_path, thumb_size=(128, 128), images_per_row=5):
    images = [Image.open(p).convert("RGB") for p in image_paths]

    # Resize images to thumbnails
    images = [img.resize(thumb_size) for img in images]

    # Calculate sheet size
    n_rows = math.ceil(len(images) / images_per_row)
    sheet_width = thumb_size[0] * images_per_row
    sheet_height = thumb_size[1] * n_rows

    # Create blank sheet
    sheet = Image.new("RGB", (sheet_width, sheet_height), color=(255,255,255))

    # Paste thumbnails
    for idx, img in enumerate(images):
        row = idx // images_per_row
        col = idx % images_per_row
        x = col * thumb_size[0]
        y = row * thumb_size[1]
        sheet.paste(img, (x, y))

    sheet.save(sheet_path)
    print(f"Contact sheet saved to {sheet_path}")


In [ ]:
# Find duplicates

## Trouver images doubles en utilisant FAISS

In [ ]:
index = faiss.read_index(os.path.join(experiment_dir, "objets_vector.index"))


In [ ]:
print(f"Index contains {index.ntotal} vectors")


Index contains 2127 vectors


In [ ]:
# import faiss
# import numpy as np
# import torch
# import os
# from tqdm import tqdm

# # === Load your vectors again (since we need them to query) ===
# embeddings = [...]  # list of paths to your .pt files
# all_vectors = []
# for path in tqdm(embeddings, desc="Loading embeddings"):
#     vec = torch.load(path)  # shape [768]
#     all_vectors.append(vec)

# all_vectors = np.stack(all_vectors).astype("float32")

# # Normalize (same as before)
# faiss.normalize_L2(all_vectors)

# # === Build FAISS index ===
# index = faiss.IndexFlatL2(all_vectors.shape[1])
# index.add(all_vectors)


# === Search for nearest neighbors ===
k = 2  # self + nearest neighbor
D, I = index.search(all_vectors, k)

all_vectors = index.reconstruct_n(0, index.ntotal)
all_vectors = np.stack(all_vectors).astype("float32")


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# === Recover stored vectors (needed for self-search) ===
all_vectors = np.stack(index.reconstruct_n(0, index.ntotal)).astype("float32")

# === Search nearest neighbors ===
k = 2  # self + nearest neighbor
D, I = index.search(all_vectors, k)

# === Detect duplicates ===
duplicates = []
threshold = 1e-7  # adjust tolerance

for i in range(len(all_vectors)):
    neighbor_idx = I[i, 1]  # closest other vector
    dist = D[i, 1]
    if dist < threshold:
        duplicates.append((i, neighbor_idx, dist))

# === Print results ===
if duplicates:
    print(f"Found {len(duplicates)} duplicates:")
    for i, j, d in duplicates:
        print(f"Vector {i} <--> Vector {j}, distance={d:.8f}")
else:
    print("No duplicates found.")

Found 53 duplicates:
Vector 68 <--> Vector 1981, distance=0.00000000
Vector 95 <--> Vector 1013, distance=0.00000000
Vector 206 <--> Vector 440, distance=0.00000000
Vector 306 <--> Vector 872, distance=0.00000000
Vector 409 <--> Vector 1077, distance=0.00000000
Vector 440 <--> Vector 440, distance=0.00000000
Vector 517 <--> Vector 865, distance=0.00000000
Vector 545 <--> Vector 1159, distance=0.00000000
Vector 552 <--> Vector 1210, distance=0.00000000
Vector 554 <--> Vector 583, distance=0.00000000
Vector 574 <--> Vector 1564, distance=0.00000000
Vector 583 <--> Vector 583, distance=0.00000000
Vector 630 <--> Vector 753, distance=0.00000000
Vector 661 <--> Vector 1450, distance=0.00000000
Vector 685 <--> Vector 720, distance=0.00000000
Vector 720 <--> Vector 720, distance=0.00000000
Vector 728 <--> Vector 1540, distance=0.00000000
Vector 742 <--> Vector 1108, distance=0.00000000
Vector 753 <--> Vector 753, distance=0.00000000
Vector 789 <--> Vector 1221, distance=0.00000000
Vector 809 

In [ ]:
import pandas as pd
import json
import os

# --- Load the JSON mapping ---
with open(os.path.join(experiment_dir, "index_map_path.json"), 'r') as f:
    index_to_path = json.load(f)

# --- Prepare duplicates DataFrame ---
duplicates_data = []

for i, j, dist in duplicates:
    file_i = os.path.basename(index_to_path[str(i)])
    file_j = os.path.basename(index_to_path[str(j)])
    duplicates_data.append({
        "idx1": i,
        "idx2": j,
        "file1": file_i,
        "file2": file_j,
        "distance": dist
    })

df_duplicates = pd.DataFrame(duplicates_data)

# --- Save to CSV if needed ---
df_duplicates.to_csv(os.path.join(experiment_dir, "duplicates.csv"), index=False)

# --- Preview ---
df_duplicates.head()

,idx1,idx2,file1,file2,distance
0,68,1981,GazetteDrouot_5424025_05_01.pt,GazetteDrouot_5424025_04_01.pt,0.0
1,95,1013,GazetteDrouot_15511693_01_01.pt,GazetteDrouot_16465688_01_01.pt,0.0
2,206,440,GazetteDrouot_21001152_03_01.pt,GazetteDrouot_17968923_03_01.pt,0.0
3,306,872,Christies_2021-09-30_296_01_02.pt,Christies_2021-09-30_296_03_02.pt,0.0
4,409,1077,GazetteDrouot_18793404_01_01.pt,GazetteDrouot_17582345_01_01.pt,0.0


# Similrity search en utilisant embeddings noir et blanc

In [ ]:
obj_embeddings_gray = os.path.join(EXPERIMENTS, 'dinov3_embeddings', 'objets_gray')

embeddings = [os.path.join(obj_embeddings_gray, e) for e in os.listdir(obj_embeddings_gray)]

obj_img_dir = '/content/drive/MyDrive/dataHN/data/processed/g-dino_obj_crops'

In [ ]:
embedding = torch.load(embeddings[0])
print(embedding.shape)

torch.Size([768])


In [ ]:
index = faiss.IndexFlatL2(768)


In [ ]:

index = faiss.IndexFlatL2(768)

all_vectors = []
index_to_path = {}

for idx, path in enumerate(tqdm(embeddings, desc="Loading embeddings")):
    vec = torch.load(path)  # shape [768]
    vec = vec.numpy().astype("float32")
    all_vectors.append(vec)

    # Save mapping
    index_to_path[idx] = path

all_vectors = np.stack(all_vectors)

faiss.normalize_L2(all_vectors)
index.add(all_vectors)

# Save FAISS index
faiss.write_index(index, os.path.join(experiment_dir, "objets_gray_vector.index"))

# Save JSON mapping
with open(os.path.join(experiment_dir, "objets_gray_index_map_path.json"), 'w') as f:
    json.dump(index_to_path, f)

Loading embeddings: 100%|██████████| 2127/2127 [01:13<00:00, 28.88it/s] 


In [ ]:
import pandas as pd
import os

TOP_K = 20  # number of top matches per image

matches = []

for i in range(len(all_vectors)):
    row_dict = {}
    query_filename = os.path.basename(index_to_path[i])
    row_dict["query"] = query_filename

    # top matches (skip self-match at index 0)
    for rank, neighbor_idx in enumerate(I[i, 1:TOP_K+1], 1):
        neighbor_filename = os.path.basename(index_to_path[neighbor_idx])
        row_dict[f"neighbor_{rank}"] = neighbor_filename
        row_dict[f"distance_{rank}"] = D[i, rank]

    matches.append(row_dict)

df_matches = pd.DataFrame(matches)

In [ ]:
df_matches.to_csv(os.path.join(experiment_dir, 'objets_gray_top20.csv'), encoding='UTF-8')